# California Traffic Collision Data from SWITRS

We use original data to generate new data about index of severity, and save it as a new table `severity.csv`.

In [41]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
# load clean data
case_ids = pd.read_csv("/Users/sunhe/Desktop/NUS_Semester1/DSA5104/project/data/clean_data/clean_case_ids.csv", low_memory=False)
collisions = pd.read_csv("/Users/sunhe/Desktop/NUS_Semester1/DSA5104/project/data/clean_data/clean_collisions.csv", low_memory=False)
victims = pd.read_csv("/Users/sunhe/Desktop/NUS_Semester1/DSA5104/project/data/clean_data/clean_victims.csv", low_memory=False)
parties = pd.read_csv("/Users/sunhe/Desktop/NUS_Semester1/DSA5104/project/data/clean_data/clean_parties.csv", low_memory=False)

In [9]:
case_ids.head()

,case_id,db_year
0,81715,2021
1,726202,2021
2,3858022,2021
3,3899441,2021
4,3899442,2021


In [10]:
collisions.head()

,case_id,jurisdiction,officer_id,reporting_district,chp_shift,population,county_city_location,county_location,special_condition,beat_type,...,bicyclist_injured_count,motorcyclist_killed_count,motorcyclist_injured_count,primary_ramp,secondary_ramp,latitude,longitude,collision_date,collision_time,process_date
0,81715,1941.0,11342,212,not chp,>250000,1941,los angeles,0,not chp,...,0,0,0.0,NaN,NaN,NaN,NaN,2020-03-14,07:45:00,2020-06-22
1,3899441,9120.0,17248,NaN,2200 thru 0559,2500 to 10000,801,del norte,0,chp state highway,...,0,0,0.0,NaN,NaN,NaN,NaN,2009-02-28,01:45:00,2009-11-02
2,3899442,9530.0,19005,NaN,0600 thru 1359,>250000,1942,los angeles,0,chp state highway,...,0,0,0.0,NaN,NaN,33.86465,-118.28533,2009-02-09,10:20:00,2010-01-14
3,3899445,9530.0,19284,NaN,1400 thru 2159,>250000,1942,los angeles,0,chp state highway,...,0,0,0.0,NaN,NaN,33.79120,-118.28230,2009-02-18,15:50:00,2010-01-13
4,3899446,9530.0,19289,NaN,1400 thru 2159,25000 to 50000,1939,los angeles,0,chp state highway,...,0,0,0.0,NaN,NaN,33.88450,-118.35260,2009-02-11,17:35:00,2010-01-11


In [11]:
victims.head()

,id,case_id,party_number,victim_role,victim_sex,victim_age,victim_degree_of_injury,victim_seating_position,victim_safety_equipment_1,victim_safety_equipment_2,victim_ejected
0,1,3858022,1,passenger,female,24.0,no injury,passenger seat 4,air bag not deployed,NaN,not ejected
1,2,3858022,1,passenger,male,31.0,no injury,passenger seat 6,air bag not deployed,NaN,not ejected
2,3,3899450,1,driver,female,72.0,complaint of pain,driver,air bag not deployed,lap/shoulder harness used,not ejected
3,4,3899453,2,driver,male,66.0,complaint of pain,driver,air bag deployed,lap/shoulder harness used,not ejected
4,5,3899453,3,passenger,male,65.0,no injury,passenger seat 3,air bag not deployed,lap/shoulder harness used,not ejected


In [12]:
parties.head()

,id,case_id,party_number,party_type,at_fault,party_sex,party_age,party_sobriety,party_drug_physical,direction_of_travel,...,other_associate_factor_2,party_number_killed,party_number_injured,movement_preceding_collision,vehicle_year,vehicle_make,statewide_vehicle_type,chp_vehicle_type_towing,chp_vehicle_type_towed,party_race
0,1,81715,1,driver,1,female,35.0,not applicable,not applicable,north,...,NaN,0,0,proceeding straight,2007.0,ford,NaN,NaN,NaN,other
1,2,81715,2,driver,0,female,43.0,not applicable,not applicable,north,...,NaN,0,0,proceeding straight,2019.0,NaN,NaN,NaN,NaN,hispanic
2,6,3899441,1,driver,1,male,16.0,impairment unknown,G,south,...,NaN,0,0,proceeding straight,1994.0,toyota,passenger car,"passenger car, station",00,asian
3,7,3899442,1,driver,1,male,40.0,had not been drinking,NaN,south,...,NaN,0,0,proceeding straight,2001.0,toyota,passenger car,"passenger car, station",00,hispanic
4,8,3899442,2,driver,0,male,46.0,had not been drinking,NaN,south,...,NaN,0,0,proceeding straight,2003.0,toyota,passenger car,"passenger car, station",00,asian


In [55]:
# generate a new table about records of severity
index1 = ['id', 'case_id', 'victim_degree_of_injury']
data1 = victims[index1]
index2 = ['case_id', 'killed_victims', 'injured_victims', 'collision_severity']
data2 = collisions[index2]
severity = pd.merge(data1, data2, left_on='case_id',right_on='case_id',how='inner')

In [56]:
severity.head()

,id,case_id,victim_degree_of_injury,killed_victims,injured_victims,collision_severity
0,3,3899450,complaint of pain,0.0,1.0,pain
1,4,3899453,complaint of pain,0.0,1.0,pain
2,5,3899453,no injury,0.0,1.0,pain
3,8,3899457,no injury,0.0,0.0,property damage only
4,9,3899457,no injury,0.0,0.0,property damage only


In [57]:
# generate count_victims by summing killed_victims and injured_victims
severity['count_victims'] = severity['killed_victims'] + severity['injured_victims']
severity.head()

,id,case_id,victim_degree_of_injury,killed_victims,injured_victims,collision_severity,count_victims
0,3,3899450,complaint of pain,0.0,1.0,pain,1.0
1,4,3899453,complaint of pain,0.0,1.0,pain,1.0
2,5,3899453,no injury,0.0,1.0,pain,1.0
3,8,3899457,no injury,0.0,0.0,property damage only,0.0
4,9,3899457,no injury,0.0,0.0,property damage only,0.0


In [22]:
severity.shape

(6868571, 7)

In [20]:
severity['victim_degree_of_injury'].unique()

array(['complaint of pain', 'no injury', 'severe injury',
       'other visible injury', 'killed', 'suspected minor injury',
       'possible injury', 'suspected serious injury'], dtype=object)

In [21]:
severity['collision_severity'].unique()

array(['pain', 'property damage only', 'severe injury', 'other injury',
       'fatal'], dtype=object)

In [35]:
map_of_victim_degree_of_injury = {
    "no injury": "0",
    "complaint of pain": "1",
    "possible injury": "2",
    "suspected minor injury": "3",
    "other visible injury": "4",
    "suspected serious injury": "5",
    "severe injury": "6",
    "killed": "7",
}
map_of_collision_severity = {
    "property damage only": "0",
    "pain": "1",
    "other injury": "2",
    "severe injury": "3",
    "fatal": "4",
}

In [36]:
labeled_severity = severity

In [39]:
# LabelEncoder of victim_degree_of_injury and collision_severity
for i in range(labeled_severity.shape[0]):
    labeled_severity['victim_degree_of_injury'][i] = map_of_victim_degree_of_injury[labeled_severity['victim_degree_of_injury'][i]]
    labeled_severity['collision_severity'][i] = map_of_collision_severity[labeled_severity['collision_severity'][i]]

/var/folders/vf/x9x39h7s5h76xttp141rxjg80000gn/T/ipykernel_61316/3054609408.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/vf/x9x39h7s5h76xttp141rxjg80000gn/T/ipykernel_61316/3054609408.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [40]:
labeled_severity.head()

,id,case_id,victim_degree_of_injury,killed_victims,injured_victims,collision_severity,count_victims
0,3,3899450,1,0.0,1.0,1,1.0
1,4,3899453,1,0.0,1.0,1,1.0
2,5,3899453,0,0.0,1.0,1,1.0
3,8,3899457,0,0.0,0.0,0,0.0
4,9,3899457,0,0.0,0.0,0,0.0


In [45]:
# use MinMaxScaler to compress data to [0,1]
scaler = MinMaxScaler()
labeled_severity['victim_degree_of_injury'] = scaler.fit_transform(labeled_severity['victim_degree_of_injury'].values.reshape(-1, 1))
labeled_severity['collision_severity'] = scaler.fit_transform(labeled_severity['collision_severity'].values.reshape(-1, 1))
labeled_severity['count_victims'] = scaler.fit_transform(labeled_severity['count_victims'].values.reshape(-1, 1))

In [46]:
# generate severity index by summing standardized victim_degree_of_injury， collision_severity and count_victims
labeled_severity['severity_index'] = labeled_severity['victim_degree_of_injury'] + labeled_severity['collision_severity'] + labeled_severity['count_victims']
labeled_severity.head()

,id,case_id,victim_degree_of_injury,killed_victims,injured_victims,collision_severity,count_victims,severity_index
0,3,3899450,0.142857,0.0,1.0,0.25,0.009524,0.402381
1,4,3899453,0.142857,0.0,1.0,0.25,0.009524,0.402381
2,5,3899453,0.000000,0.0,1.0,0.25,0.009524,0.259524
3,8,3899457,0.000000,0.0,0.0,0.00,0.000000,0.000000
4,9,3899457,0.000000,0.0,0.0,0.00,0.000000,0.000000


In [58]:
severity['severity_index'] = labeled_severity['severity_index']
severity.head()

,id,case_id,victim_degree_of_injury,killed_victims,injured_victims,collision_severity,count_victims,severity_index
0,3,3899450,complaint of pain,0.0,1.0,pain,1.0,0.402381
1,4,3899453,complaint of pain,0.0,1.0,pain,1.0,0.402381
2,5,3899453,no injury,0.0,1.0,pain,1.0,0.259524
3,8,3899457,no injury,0.0,0.0,property damage only,0.0,0.000000
4,9,3899457,no injury,0.0,0.0,property damage only,0.0,0.000000


In [61]:
severity.shape

(6868571, 8)

In [60]:
# save new data
with open('/Users/sunhe/Desktop/NUS_Semester1/DSA5104/project/data/clean_data/severity.csv',
          'a', encoding='utf8', newline="") as f:
    severity.to_csv(f, header=True, index=0)